In [1]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from sub import subMNIST

Files already downloaded
3000
750


In [2]:
trainset_labeled = pickle.load(open("train_labeled.p", "rb")) 
train_loader  = torch.utils.data.DataLoader(trainset_labeled, batch_size=64, shuffle=True, num_workers=2)

validset = pickle.load(open("validation.p", "rb"))
valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True, num_workers=2)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)

model = Net()
params = list(model.parameters())

print(model)
print('Models has {} learnable paramater:'.format(len(params)))
[print('parameter {} has a size of {}'.format(i+1, params[i].size())) for i in range(len(params))]

Net (
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d (p=0.5)
  (fc1): Linear (320 -> 50)
  (fc2): Linear (50 -> 10)
)
Models has 8 learnable paramater:
parameter 1 has a size of torch.Size([10, 1, 5, 5])
parameter 2 has a size of torch.Size([10])
parameter 3 has a size of torch.Size([20, 10, 5, 5])
parameter 4 has a size of torch.Size([20])
parameter 5 has a size of torch.Size([50, 320])
parameter 6 has a size of torch.Size([50])
parameter 7 has a size of torch.Size([10, 50])
parameter 8 has a size of torch.Size([10])


[None, None, None, None, None, None, None, None]

In [4]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [5]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target) # Wrap them in Variable 
        optimizer.zero_grad() # Zero the parameter gradients
        output = model(data) # Forward 
        loss = F.nll_loss(output, target) 
        loss.backward() 
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [6]:
def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [8]:
for epoch in range(1, 11):
    train(epoch)
    test(epoch, valid_loader)

Train Epoch: 1 [0/3000 (0%)]	Loss: 0.958099
Train Epoch: 1 [640/3000 (21%)]	Loss: 1.151651
Train Epoch: 1 [1280/3000 (43%)]	Loss: 1.037153
Train Epoch: 1 [1920/3000 (64%)]	Loss: 1.252772
Train Epoch: 1 [2560/3000 (85%)]	Loss: 0.965604

Test set: Average loss: 0.6620, Accuracy: 8292/10000 (83%)

Train Epoch: 2 [0/3000 (0%)]	Loss: 0.899747
Train Epoch: 2 [640/3000 (21%)]	Loss: 0.936787
Train Epoch: 2 [1280/3000 (43%)]	Loss: 1.183522
Train Epoch: 2 [1920/3000 (64%)]	Loss: 1.074017
Train Epoch: 2 [2560/3000 (85%)]	Loss: 0.919468

Test set: Average loss: 0.6097, Accuracy: 8419/10000 (84%)

Train Epoch: 3 [0/3000 (0%)]	Loss: 0.965402
Train Epoch: 3 [640/3000 (21%)]	Loss: 0.992506
Train Epoch: 3 [1280/3000 (43%)]	Loss: 0.878901
Train Epoch: 3 [1920/3000 (64%)]	Loss: 0.967720
Train Epoch: 3 [2560/3000 (85%)]	Loss: 0.782421

Test set: Average loss: 0.5669, Accuracy: 8571/10000 (86%)

Train Epoch: 4 [0/3000 (0%)]	Loss: 0.834006
Train Epoch: 4 [640/3000 (21%)]	Loss: 0.881821
Train Epoch: 4 [1280/